In [ ]:
# default_exp callback.TSBERT

# TSBERT: Self-supervised Pre-training of Time Series Models

> Callback used to perform the autoregressive task of denoising the input after a binary mask has been applied.

This is an unofficial PyTorch implementation created by Ignacio Oguiza (timeseriesAI@gmail.com) based on:.

* Zerveas, G., Jayaraman, S., Patel, D., Bhamidipaty, A., & Eickhoff, C. (2020). [A Transformer-based Framework for Multivariate Time Series Representation Learning. arXiv preprint arXiv:2010.02803v2.](https://arxiv.org/pdf/2010.02803). No official implementation available as far as I know (Oct 10th, 2020)

* Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). [Attention is all you need. In Advances in neural information processing systems (pp. 5998-6008).](https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf)

* He, R., Ravula, A., Kanagal, B., & Ainslie, J. (2020). Realformer: Transformer Likes Informed Attention. arXiv preprint arXiv:2012.11747.

In [ ]:
#export
from fastai.callback.all import *
from tsai.imports import *
from tsai.utils import *
from tsai.models.utils import *
from tsai.models.layers import *

In [ ]:
#export
def create_subsequence_mask(o, r=.15, lm=3, stateful=True, sync=False):
    n_masks, mask_dims, mask_len = o.shape
    if sync == 'random': sync = random.random() > .5
    dims = 1 if sync else mask_dims
    numels = n_masks * dims * mask_len
    pm = 1 / lm
    pu = pm * (r / (1 - r))
    a, b, proba_a, proba_b = ([1], [0], pu, pm) if random.random() > pm else ([0], [1], pm, pu)
    if stateful: 
        max_len = max(1, 2 * math.ceil(numels // (1/pm + 1/pu)))
        while True:
            dist_a = np.clip(np.random.geometric(proba_a, max_len), 1, mask_len)
            dist_b = np.clip(np.random.geometric(proba_b, max_len), 1, mask_len)
            if (dist_a + dist_b).sum() >= numels:
                dist_len = np.argmax((dist_a + dist_b).cumsum() >= numels) + 1
                break
        l = [a*ax + b*bx for (ax, bx) in zip(dist_a[:dist_len], dist_b[:dist_len])]
        _mask = list(itertools.chain.from_iterable(l))[:numels]
    else: 
        _mask = np.random.binomial(1, 1 - r, numels)
    mask = torch.Tensor(_mask).reshape(n_masks, dims, mask_len)
    if sync: mask = mask.repeat(1, mask_dims, 1)
    return mask.to(o.device)

def create_variable_mask(o, r=.15):
    n_masks, mask_dims, mask_len = o.shape
    sel_dims = np.random.choice(n_masks * mask_dims, int(n_masks * mask_dims * r), False)
    _mask = np.ones((n_masks * mask_dims, mask_len))
    _mask[sel_dims] = 0
    mask = torch.Tensor(_mask).reshape(*o.shape)
    return mask.to(o.device)

In [ ]:
t = torch.rand(16, 3, 100)
mask = create_subsequence_mask(t, sync=False)
test_eq(mask.shape, t.shape)
mask = create_subsequence_mask(t, sync=True)
test_eq(mask.shape, t.shape)
mask = create_variable_mask(t)
test_eq(mask.shape, t.shape)

In [ ]:
#export
class TSBERT_Loss(Module):
    def __init__(self, crit=None):
        self.crit = ifnone(crit, MSELossFlat())
        self.mask = slice(None)
        
    def forward(self, preds, target):
        return self.crit(preds[self.mask], target[self.mask])

In [ ]:
# export
class TSBERT(Callback):
    def __init__(self, r:float=.15, subsequence_mask:bool=True, lm:float=3., stateful:bool=True, sync:bool=False, variable_mask:bool=False,
                 crit:callable=None, target_dir:str='./data/TSBERT', fname:str='ssd_trained_model',verbose:bool=True):
        r"""
        A callback to implement a self-supervised denoising task.
        Args:
            mask_shape: expected mask shape (tuple). e.g batch size, n_vars, timesteps
            r: proba of masking.
            subsequence_mask: apply a mask to random subsequences.
            lm: average mask len when using stateful (geometric) masking.
            stateful: geometric distribution is applied so that average mask length is lm.
            sync: all variables have the same masking.
            variable_mask: apply a mask to random variables.
            crit: loss function that will be used. If None MSELossFlat().
            target_dir : directory where trained model will be stored.
            fname : name that will be used to save the pretrained model.
    """
        self.subsequence_mask,self.variable_mask=subsequence_mask,variable_mask
        assert self.subsequence_mask or self.variable_mask, 'you must set subsequence_mask and/or variable_mask to True'
        self.r,self.lm,self.stateful,self.sync,self.crit,self.target_dir,self.fname,self.verbose = r,lm,stateful,sync,crit,Path(target_dir),fname,verbose
        
    def before_fit(self):
        # modify loss for denoising task
        self.old_loss_func = self.learn.loss_func
        self.learn.loss_func = TSBERT_Loss(self.crit)
        
        # save initial model head
        assert hasattr(self.learn.model, "head"), "you can only use SSD with models that have .head attributes"
        backbone, head = split_model(self.learn.model)
        self.head = head
        self.learn.model = backbone
        
        # prepare model for denoising task
        with torch.no_grad():
            b = self.learn.dls.train.one_batch()
            ni = b[0].shape[1] 
            out = self.learn.model(b[0])
            assert out.ndim == 3, "make sure the backbone (after model.head = Noop) produces a 3d output [bs x nf x seq_len]"
            no = out.shape[1]
            self.learn.model.head = nn.Sequential(*[Transpose(-1, -2), nn.Linear(no, ni), Transpose(-1, -2)]).to(b[0].device)
            assert self.learn.model(b[0]).shape == b[0].shape
    
    def before_batch(self):
        if self.subsequence_mask and self.variable_mask:
            random_thr = 1/3 if self.sync == 'random' else 1/2
            if random.random() > random_thr: 
                mask = create_subsequence_mask(self.x, r=self.r, lm=self.lm, stateful=self.stateful, sync=self.sync)
            else: 
                mask = create_variable_mask(self.x, r=self.r)
        elif self.subsequence_mask:
            mask = create_subsequence_mask(self.x, r=self.r, lm=self.lm, stateful=self.stateful, sync=self.sync)
        else: 
            mask = create_variable_mask(self.x, r=self.r)
        self.learn.yb = (self.x,)
        self.learn.xb = (self.x * mask,)
        self.learn.loss_func.mask = mask == 0

    def after_fit(self):
        if self.epoch == self.n_epoch - 1 and not "LRFinder" in [cls_name(cb) for cb in self.learn.cbs]:
            self.learn.loss_func = self.old_loss_func
            self.learn.model.head = self.head
            PATH = Path(f'{self.target_dir/self.fname}.pth')
            if not os.path.exists(PATH.parent): os.makedirs(PATH.parent)
            torch.save(self.learn.model.state_dict(), PATH)
            pv(f"\npre-trained model weights path: {PATH}\n", self.verbose)
        self.learn.remove_cb(self)

# Experiments

In [ ]:
from fastai.data.transforms import *
from tsai.data.all import *
from tsai.models.utils import *
from tsai.models.layers import *
from tsai.learner import *
from tsai.models.TSTPlus import *
from tsai.models.InceptionTimePlus import *

In [ ]:
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, split_data=False)
check_data(X, y, splits)

X      - shape: [360 samples x 24 features x 51 timesteps]  type: memmap  dtype:float32  isnan: 0
y      - shape: (360,)  type: memmap  dtype:<U3  n_classes: 6 (60 samples per class) ['1.0', '2.0', '3.0', '4.0', '5.0', '6.0']  isnan: False
splits - n_splits: 2 shape: [180, 180]  overlap: [False]


In [ ]:
# Pre-train
tfms  = [None, [Categorize()]]
batch_tfms = [TSStandardize(by_var=True)]
unlabeled_dls = get_ts_dls(X, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=64)
tsbert = TSBERT(subsequence_mask=True, sync='random', variable_mask=True, fname=f'{dsid}_test')
learn = ts_learner(unlabeled_dls, InceptionTimePlus, cbs=tsbert)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,time
0,1.531935,1.087306,00:05



pre-trained model weights path: data/TSBERT/NATOPS_test.pth



In [ ]:
# Fine-tune
tfms  = [None, [Categorize()]]
batch_tfms = [TSStandardize(by_var=True)]
labeled_dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=64)
learn = ts_learner(labeled_dls, InceptionTimePlus, pretrained=True, weights_path='data/TSBERT/NATOPS_test.pth', metrics=[accuracy])
learn.fine_tune(1)

weights from data/TSBERT/NATOPS_test.pth successfully transferred!



epoch,train_loss,valid_loss,accuracy,time
0,1.897716,1.774922,0.161111,00:05


epoch,train_loss,valid_loss,accuracy,time
0,1.121014,1.736586,0.177778,00:06


In [ ]:
# hide
out = create_scripts(); beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 000b_data.validation.ipynb.
Converted 000c_data.preparation.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.preprocessing.ipynb.
Converted 003b_data.transforms.ipynb.
Converted 003c_data.mixed_augmentation.ipynb.
Converted 003d_data.image.ipynb.
Converted 003e_data.features.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.mixed.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_callback.core.ipynb.
Converted 011_callback.noisy_student.ipynb.
Converted 012_callback.gblend.ipynb.
Converted 013_callback.TSBERT.ipynb.
Converted 100_models.utils.ipynb.
Converted 100b_models.layers.ipynb.
Converted 100c_models.explainability.ipynb.
Converted 101_models.ResNet.ipynb.
Converted 101b_models.ResNetPlus.ipynb.
Converted 102_models.InceptionTime.ipynb.
Converted 102b_models.InceptionTimePlus.ipynb.
Converted 103_models.MLP.ipynb.
Converted 103b_mode